# Manual Metrics Computation Tutorial

This notebook teaches you how to manually compute object detection metrics:
- **Precision & Recall**
- **IoU (Intersection over Union)**
- **AP (Average Precision)**
- **mAP (mean Average Precision)**

We'll use the validation results from your trained models and compute metrics **with and without background class** to understand the difference.

## 1. Load Validation Results

First, let's load the validation metrics that were saved during training.

In [ ]:
import json
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Load the validation metrics from a trained model
model_dir = Path('../experiments/model_comparison/retinanet_resnet50')
metrics_file = model_dir / 'validation' / 'validation_metrics.json'

with open(metrics_file, 'r') as f:
    metrics = json.load(f)

print("Loaded metrics from:", metrics_file)
print("\nAvailable keys:", list(metrics.keys()))
print("\nTop-level metrics:")
for key in ['mAP@0.5', 'mAP@0.5:0.95', 'precision', 'recall', 'f1_score']:
    print(f"  {key}: {metrics[key]:.4f}")

## 2. Understanding Per-Class Metrics

Let's look at the per-class Average Precision (AP) values.

In [ ]:
print("\nPer-class metrics:")
print(f"{'Class':<10} {'AP@0.5':<12} {'Precision':<12} {'Recall':<12}")
print("-" * 50)

class_aps = []
class_names_mapping = {
    0: "Background",
    1: "bien_bao_hieu_lenh",
    2: "bien_bao_nguy_hiem",
    3: "bien_chi_dan",
    4: "bien_phu",
    5: "bien_bao_cam"
}

for class_key, class_data in metrics['per_class'].items():
    class_id = int(class_key.split('_')[1])
    ap = class_data['ap']
    precision = class_data['precision']
    recall = class_data['recall']
    
    class_name = class_names_mapping.get(class_id, f"Class {class_id}")
    print(f"{class_name:<10} {ap:<12.4f} {precision:<12.4f} {recall:<12.4f}")
    class_aps.append((class_id, ap))

print("\n⚠️ Notice: Class 0 (Background) has AP = 0.0000")
print("This is WRONG because background shouldn't be included in evaluation!")

## 3. Manual mAP Computation - INCORRECT (With Background)

Let's manually compute mAP the **wrong way** (including background):

In [ ]:
print("\n" + "="*70)
print("INCORRECT mAP Computation (Including Background)")
print("="*70)

# Extract all AP values (including class 0)
all_aps = [class_data['ap'] for class_data in metrics['per_class'].values()]

print("\nAP values for each class:")
for i, ap in enumerate(all_aps):
    print(f"  Class {i}: {ap:.4f}")

# Compute mean
incorrect_mAP = np.mean(all_aps)

print(f"\nFormula: mAP = (AP_0 + AP_1 + AP_2 + AP_3 + AP_4 + AP_5) / 6")
print(f"mAP = ({' + '.join([f'{ap:.4f}' for ap in all_aps])}) / {len(all_aps)}")
print(f"mAP = {sum(all_aps):.4f} / {len(all_aps)}")
print(f"\n❌ Incorrect mAP@0.5 = {incorrect_mAP:.4f}")
print(f"   (This matches the reported value: {metrics['mAP@0.5']:.4f})")

print("\n⚠️ Problem: Class 0 (background) with AP=0.0 artificially lowers the mAP!")

## 4. Manual mAP Computation - CORRECT (Excluding Background)

Now let's compute mAP the **correct way** (excluding background class):

In [ ]:
print("\n" + "="*70)
print("CORRECT mAP Computation (Excluding Background)")
print("="*70)

# Extract AP values for classes 1-5 only (excluding class 0)
correct_aps = []
for class_key, class_data in metrics['per_class'].items():
    class_id = int(class_key.split('_')[1])
    if class_id > 0:  # Skip background (class 0)
        correct_aps.append(class_data['ap'])

print("\nAP values for actual classes (excluding background):")
for i, ap in enumerate(correct_aps, start=1):
    print(f"  Class {i}: {ap:.4f}")

# Compute mean
correct_mAP = np.mean(correct_aps)

print(f"\nFormula: mAP = (AP_1 + AP_2 + AP_3 + AP_4 + AP_5) / 5")
print(f"mAP = ({' + '.join([f'{ap:.4f}' for ap in correct_aps])}) / {len(correct_aps)}")
print(f"mAP = {sum(correct_aps):.4f} / {len(correct_aps)}")
print(f"\n✅ Correct mAP@0.5 = {correct_mAP:.4f}")

# Show improvement
improvement = ((correct_mAP - incorrect_mAP) / incorrect_mAP) * 100
print(f"\n📈 Improvement: +{improvement:.1f}% ({correct_mAP:.4f} vs {incorrect_mAP:.4f})")
print(f"   Absolute gain: +{correct_mAP - incorrect_mAP:.4f}")

## 5. Understanding IoU (Intersection over Union)

IoU measures the overlap between predicted and ground truth bounding boxes.

**Formula:**
```
IoU = Area of Intersection / Area of Union
```

Let's implement and visualize it:

In [ ]:
def compute_iou(box1, box2):
    """
    Compute IoU between two bounding boxes
    
    Args:
        box1, box2: [x1, y1, x2, y2] format
        
    Returns:
        IoU value (0 to 1)
    """
    # Get coordinates
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    
    # Compute intersection
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    # Check if boxes overlap
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    # Compute areas
    intersection_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - intersection_area
    
    # Compute IoU
    iou = intersection_area / union_area
    
    return iou


# Example boxes
print("\nIoU Examples:")
print("="*70)

# Example 1: Perfect overlap
box_gt = [10, 10, 50, 50]
box_pred = [10, 10, 50, 50]
iou = compute_iou(box_gt, box_pred)
print(f"\n1. Perfect overlap:")
print(f"   Ground Truth: {box_gt}")
print(f"   Prediction:   {box_pred}")
print(f"   IoU = {iou:.4f} (perfect match!)")

# Example 2: Partial overlap
box_pred = [20, 20, 60, 60]
iou = compute_iou(box_gt, box_pred)
print(f"\n2. Partial overlap:")
print(f"   Ground Truth: {box_gt}")
print(f"   Prediction:   {box_pred}")
print(f"   IoU = {iou:.4f}")

# Example 3: Small overlap
box_pred = [40, 40, 80, 80]
iou = compute_iou(box_gt, box_pred)
print(f"\n3. Small overlap:")
print(f"   Ground Truth: {box_gt}")
print(f"   Prediction:   {box_pred}")
print(f"   IoU = {iou:.4f}")

# Example 4: No overlap
box_pred = [100, 100, 140, 140]
iou = compute_iou(box_gt, box_pred)
print(f"\n4. No overlap:")
print(f"   Ground Truth: {box_gt}")
print(f"   Prediction:   {box_pred}")
print(f"   IoU = {iou:.4f}")

print("\n💡 Key: IoU > 0.5 is typically considered a 'correct' detection")

## 6. Computing Precision and Recall

For object detection:

**Precision** = TP / (TP + FP)  
**Recall** = TP / (TP + FN)

Where:
- **TP (True Positive)**: Correct detection (IoU ≥ threshold)
- **FP (False Positive)**: Incorrect detection (IoU < threshold or duplicate)
- **FN (False Negative)**: Missed ground truth

Let's compute these manually:

In [ ]:
print("\n" + "="*70)
print("Manual Precision & Recall Computation")
print("="*70)

# Example scenario for one class
print("\n📋 Scenario: Detecting traffic signs in 5 images")
print("\nGround Truth:")
print("  Image 1: 2 signs")
print("  Image 2: 1 sign")
print("  Image 3: 3 signs")
print("  Image 4: 0 signs")
print("  Image 5: 1 sign")
print("  Total GT: 7 signs")

print("\nPredictions (with IoU check):")
print("  Image 1: 2 detections (both IoU > 0.5) → 2 TP")
print("  Image 2: 1 detection (IoU > 0.5) → 1 TP")
print("  Image 3: 4 detections (2 with IoU > 0.5, 1 with IoU < 0.5, 1 duplicate) → 2 TP, 2 FP")
print("  Image 4: 1 detection (no GT, so IoU = 0) → 1 FP")
print("  Image 5: 0 detections → 0 TP (1 FN)")

# Count detections
TP = 2 + 1 + 2 + 0 + 0  # True Positives
FP = 0 + 0 + 2 + 1 + 0  # False Positives
FN = 0 + 0 + 1 + 0 + 1  # False Negatives (missed GT)
total_GT = 7

print(f"\n\n📊 Counts:")
print(f"  True Positives (TP):  {TP}")
print(f"  False Positives (FP): {FP}")
print(f"  False Negatives (FN): {FN}")
print(f"  Total Ground Truth:   {total_GT}")

# Compute metrics
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\n\n📈 Metrics:")
print(f"  Precision = TP / (TP + FP) = {TP} / ({TP} + {FP}) = {TP} / {TP + FP} = {precision:.4f}")
print(f"  Recall    = TP / (TP + FN) = {TP} / ({TP} + {FN}) = {TP} / {TP + FN} = {recall:.4f}")
print(f"  F1 Score  = 2 * (P * R) / (P + R) = 2 * ({precision:.4f} * {recall:.4f}) / ({precision:.4f} + {recall:.4f}) = {f1:.4f}")

print("\n💡 Interpretation:")
print(f"  - {precision*100:.1f}% of detections are correct")
print(f"  - We found {recall*100:.1f}% of all ground truth signs")

## 7. Computing Average Precision (AP)

AP summarizes the precision-recall curve. Here's how to compute it:

1. Sort predictions by confidence score (highest first)
2. For each prediction, compute precision and recall
3. Plot precision-recall curve
4. Compute area under curve (AUC)

In [ ]:
def compute_ap(precisions, recalls):
    """
    Compute Average Precision using 11-point interpolation
    
    Args:
        precisions: Array of precision values
        recalls: Array of recall values
        
    Returns:
        Average Precision
    """
    # 11-point interpolation
    ap = 0.0
    for threshold in np.linspace(0, 1, 11):
        # Get precision at recall >= threshold
        precisions_at_recall = precisions[recalls >= threshold]
        if len(precisions_at_recall) > 0:
            ap += np.max(precisions_at_recall)
    
    return ap / 11.0


print("\n" + "="*70)
print("Computing Average Precision (AP)")
print("="*70)

# Example: Predictions for one class sorted by confidence
predictions = [
    {'confidence': 0.95, 'iou': 0.8, 'correct': True},   # TP
    {'confidence': 0.90, 'iou': 0.7, 'correct': True},   # TP
    {'confidence': 0.85, 'iou': 0.3, 'correct': False},  # FP
    {'confidence': 0.80, 'iou': 0.6, 'correct': True},   # TP
    {'confidence': 0.75, 'iou': 0.2, 'correct': False},  # FP
    {'confidence': 0.70, 'iou': 0.5, 'correct': True},   # TP
]
total_gt = 6  # Total ground truth objects

print("\nPredictions (sorted by confidence):")
print(f"{'#':<4} {'Conf':<8} {'IoU':<8} {'Type':<8} {'TP':<6} {'FP':<6} {'Precision':<12} {'Recall':<12}")
print("-" * 75)

tp_cumsum = 0
fp_cumsum = 0
precisions_list = []
recalls_list = []

for i, pred in enumerate(predictions, 1):
    if pred['correct']:
        tp_cumsum += 1
        pred_type = 'TP'
    else:
        fp_cumsum += 1
        pred_type = 'FP'
    
    precision = tp_cumsum / (tp_cumsum + fp_cumsum)
    recall = tp_cumsum / total_gt
    
    precisions_list.append(precision)
    recalls_list.append(recall)
    
    print(f"{i:<4} {pred['confidence']:<8.2f} {pred['iou']:<8.2f} {pred_type:<8} "
          f"{tp_cumsum:<6} {fp_cumsum:<6} {precision:<12.4f} {recall:<12.4f}")

# Compute AP
precisions_array = np.array(precisions_list)
recalls_array = np.array(recalls_list)
ap = compute_ap(precisions_array, recalls_array)

print(f"\n📊 Average Precision (AP) = {ap:.4f}")

# Plot precision-recall curve
plt.figure(figsize=(10, 6))
plt.plot(recalls_list, precisions_list, 'b-o', linewidth=2, markersize=8, label='Precision-Recall curve')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title(f'Precision-Recall Curve (AP = {ap:.4f})', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.legend(fontsize=10)

# Shade area under curve
plt.fill_between(recalls_list, precisions_list, alpha=0.2)

plt.tight_layout()
plt.show()

print("\n💡 The area under this curve is the Average Precision (AP)")

## 8. Visualize mAP Comparison

Let's visualize the difference between correct and incorrect mAP computation:

In [ ]:
# Extract class APs
class_names = ['Background', 'bien_bao_hieu_lenh', 'bien_bao_nguy_hiem', 'bien_chi_dan', 'bien_phu', 'bien_bao_cam']
class_aps_with_bg = [metrics['per_class'][f'class_{i}']['ap'] for i in range(6)]
class_aps_without_bg = class_aps_with_bg[1:]  # Exclude class 0

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left: With background
x1 = np.arange(len(class_aps_with_bg))
colors1 = ['red'] + ['green'] * 5
bars1 = ax1.bar(x1, class_aps_with_bg, color=colors1, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Class ID', fontsize=12)
ax1.set_ylabel('Average Precision (AP)', fontsize=12)
ax1.set_title(f'❌ INCORRECT: mAP with Background\nmAP@0.5 = {incorrect_mAP:.4f}', fontsize=14)
ax1.set_xticks(x1)
ax1.set_xticklabels([f'{i}' for i in range(6)])
ax1.axhline(y=incorrect_mAP, color='blue', linestyle='--', linewidth=2, label=f'mAP = {incorrect_mAP:.4f}')
ax1.set_ylim([0, 1.05])
ax1.grid(axis='y', alpha=0.3)
ax1.legend(fontsize=10)

# Add value labels on bars
for bar, val in zip(bars1, class_aps_with_bg):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{val:.3f}', ha='center', va='bottom', fontsize=10)

# Annotate background class
ax1.text(0, -0.15, '← Background\n(should not\nbe included!)', ha='center', fontsize=10, color='red', weight='bold')

# Right: Without background
x2 = np.arange(len(class_aps_without_bg))
bars2 = ax2.bar(x2, class_aps_without_bg, color='green', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Class ID (remapped)', fontsize=12)
ax2.set_ylabel('Average Precision (AP)', fontsize=12)
ax2.set_title(f'✅ CORRECT: mAP without Background\nmAP@0.5 = {correct_mAP:.4f}', fontsize=14)
ax2.set_xticks(x2)
ax2.set_xticklabels([f'{i}' for i in range(5)])
ax2.axhline(y=correct_mAP, color='blue', linestyle='--', linewidth=2, label=f'mAP = {correct_mAP:.4f}')
ax2.set_ylim([0, 1.05])
ax2.grid(axis='y', alpha=0.3)
ax2.legend(fontsize=10)

# Add value labels on bars
for bar, val in zip(bars2, class_aps_without_bg):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{val:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('../experiments/map_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📈 Improvement by excluding background: +{improvement:.1f}%")
print(f"   From {incorrect_mAP:.4f} to {correct_mAP:.4f}")
print(f"\n💾 Plot saved to: experiments/map_comparison.png")

## 9. Key Takeaways

### ✅ Correct Metrics Computation:

1. **Filter background class**: Remove predictions with label = 0
2. **Remap labels**: Convert 1→0, 2→1, 3→2, etc.
3. **Compute metrics over N-1 classes**: If model has N classes (including background), compute mAP over N-1 classes

### 📊 Formulas Summary:

```python
# IoU
IoU = Intersection_Area / Union_Area

# Precision & Recall
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1 = 2 * (Precision * Recall) / (Precision + Recall)

# Average Precision (AP)
AP = Area_Under_Precision_Recall_Curve

# mean Average Precision (mAP)
mAP = (AP_class1 + AP_class2 + ... + AP_classN) / N
# Note: N should NOT include background class!
```

### 🎯 Your Models' True Performance:

Run the `recompute_metrics.py` script to get corrected metrics for all your trained models!